In [ ]:
from __future__ import print_function
from future_builtins import map, filter, zip

In [ ]:
import tensorflow as tf
import keras
import coremltools
import numpy as np

for name, package in [('tensorflow',tf), ('keras',keras), ('coremltools',coremltools), ('numpy', np)]:
    try:
        print(name + ' v' + package.__version__)
    except AttributeError:
        print(name + ' v?')

In [ ]:
# Load newline-delimited list of categories

# f = open('quickdraw-categories.txt','r')
f = open('50-categories.txt','r')
categories = [line.rstrip() for line in f]
f.close()

filename_from_category = lambda s: '../../quickdraw/'+s+'.npy'
filenames = list(map(filename_from_category, categories))

In [ ]:
# Test that all files exist according to categories
# in quickdraw-categories.txt

import os.path

all_exist = True
for filename in filenames:
    if not os.path.isfile(filename):
        print('file `{}` does not exist'.format(filename))
        all_exist = False
if all_exist:
    print('All {} files found!'.format(len(categories)))

In [ ]:
image_size = (28,28)
samples_per_category = 25000

In [ ]:
def one_hot_vector(index):
    hot = np.zeros(len(categories),dtype=np.int8)
    hot[index] = 1
    return hot

def one_hot_array(index):
    is_column = False
    hot = np.zeros((1,len(categories)),dtype=np.int8)
    hot[0,index] = 1
    if is_column:
        return hot.T
    else:
        return hot

In [ ]:
import random

def dataset_generator():
    categories_and_filenames = list(zip(categories, filenames))
    # Shuffle order
    random.shuffle(categories_and_filenames)
    for category_index, (category, filename) in enumerate(categories_and_filenames):
        label_one_hot_encoded = one_hot_array(category_index)
        category_images = np.load(filename)
        n_images = category_images.shape[0]
        sample_indices = random.sample(range(n_images), min(samples_per_category,n_images))
        for image in category_images[sample_indices]:
            image = np.matrix(image)
            image = np.expand_dims(image, axis=1)
            yield (image, label_one_hot_encoded)
        yield None

In [ ]:
import random

def make_dataset():
    data = []
    target = []
    for category_index, (category, filename) in enumerate(zip(categories, filenames)):
        label_one_hot_encoded = one_hot_vector(category_index)
        category_images = np.load(filename)
        n_images = category_images.shape[0]
        for image in category_images[:samples_per_category]:
#             image = np.matrix(image)
            data.append(image)
            target.append(label_one_hot_encoded)
    data = np.array(data)
    data = np.expand_dims(data, axis=1)
    target = np.array(target)
    print(data.shape)
    print(target.shape)
    return (data, target)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Reshape, Dense, BatchNormalization, Dropout, \
                         Conv2D, MaxPooling2D, Activation, Flatten, Lambda

### PARAMS ###
input_shape=image_size
n_filters=[32,64,64,256]
dense_sizes=[128,128]
output_size=len(categories)

batch_normalize = True

layer_activation='relu'
final_activation='softmax'

loss='categorical_crossentropy'
optimizer='rmsprop'
metrics=['categorical_accuracy']

def _make_convolution_layers(model):
    model.add(
        Reshape(
            (1, input_shape[0], input_shape[1]), 
            input_shape=(1, input_shape[0]*input_shape[1])
        )
    )
    
    model.add(
        Lambda(lambda x: x/127.5-1)
    )

    for filters in n_filters:
        # Add a single convolution layer
        model.add(
            Conv2D(
                filters=filters,
                kernel_size=(3,3),
                padding='same',
                activation=layer_activation
            )
        )
        # Add batch normalization to the convolution layer
        if batch_normalize:
            model.add(
                BatchNormalization(axis=1)
            )
        # Pool the layer
        model.add(
            MaxPooling2D(pool_size=(2,2), data_format='channels_first')
        )

    model.add(Flatten())

def _make_dense_layers(model):
    for size in dense_sizes:
        model.add(
            Dense(size, activation=layer_activation)
        )
        if batch_normalize:
            model.add(
                BatchNormalization(axis=1)
            )
    model.add(
        Dense(output_size, activation=final_activation)
    )

def one_hot_output(y_hat_raw):
    # predict() outputs floats. We want one-hot in most cases
    y_hat = np.zeros(y_hat_raw.shape, dtype=np.int)
    y_hat[np.arange(y_hat_raw.shape[0]), y_hat_raw.argmax(1)] = 1
    return y_hat

In [ ]:
data, target = make_dataset()

In [ ]:
import os.path
from keras.models import load_model
import json

def cache_fit(model_name, model, *args, **kwargs):
    archive_name = model_name+'_model.h5'
    history_name = model_name+'_history.json'
    archive_exists = os.path.isfile(archive_name)

    if not archive_exists:
        print('Model '+model_name+' not found in archive. Training new model.')
        hist = model.fit(*args, **kwargs)
        model.save(archive_name)
        with open(history_name, 'w') as f:
            json.dump(hist.history, f)
        return model
    else:
        print('Model found on disk. Reloading.')
        return load_model(archive_name)

In [ ]:
model = Sequential()

_make_convolution_layers(model)
_make_dense_layers(model)

model.compile(
    loss=loss,
    optimizer=optimizer,
    metrics=metrics
)

model = cache_fit(
    'cnn_50cat_25000img_10ep', model, 
    x=data, y=target, 
    batch_size=32, epochs=10, 
    verbose=1, shuffle=True
)